In [46]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load class names
class_names_load_path = '/content/drive/MyDrive/ML/class_names.pkl'

with open(class_names_load_path, 'rb') as f:
    loaded_class_names = pickle.load(f)

print(f"Class names loaded: {loaded_class_names}")

In [ ]:
# Load the trained model
model_load_path = '/content/drive/MyDrive/ML/trained_model1.keras'
loaded_model = keras.models.load_model(model_load_path)

loaded_model.summary()

In [ ]:
# Load test dataset
test_dir = '/content/drive/MyDrive/ML/Data/Test/'

input_shape = (500, 500, 3)

test_data = keras.Sequential([
    layers.Rescaling(1./255)
])

BATCH_SIZE = 32
test_ds = keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(input_shape[0], input_shape[1]),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = test_ds.map(lambda x, y: (test_data(x, training=False), y))

print(f"Number of test batches: {tf.data.experimental.cardinality(test_ds)}")

In [ ]:
# Evaluate
loss, accuracy = loaded_model.evaluate(test_ds)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
# Predictions
predictions = loaded_model.predict(test_ds)

# Maximum probability for each prediction
predicted_classes = tf.argmax(predictions, axis=1).numpy()

print("First 5 predicted class indices:", predicted_classes[:5])
print("First 5 predicted class names:", predicted_class_names[:5])

In [ ]:
# Test visualize
print("Test visualize class names:", predicted_class_names[:5])

In [ ]:
true_labels = []
for images, labels in test_ds:
    true_labels.append(labels.numpy())

true_labels = np.concatenate(true_labels)

print("Shape of true labels:", true_labels.shape)
print("First 10 true labels (one-hot encoded):", true_labels[:10])

In [ ]:
true_labels_indices = tf.argmax(true_labels, axis=1).numpy()

print("Shape of true label indices:", true_labels_indices.shape)
print("First 10 true label indices:", true_labels_indices[:10])

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Sample images with class labels and predictions
plt.figure(figsize=(8, 8))

for i, (images, labels) in enumerate(test_ds.take(1)):
    for j in range(min(images.shape[0], 4)): # 4 images
        ax = plt.subplot(2, 2, j + 1)
        plt.imshow(images[j].numpy())

        true_label_index = tf.argmax(labels[j]).numpy()

        overall_image_index = i * images.shape[0] + j
        predicted_label_index = predicted_classes[overall_image_index]
        image_predictions = predictions[overall_image_index]

        true_class_name = loaded_class_names[true_label_index]
        predicted_class_name = loaded_class_names[predicted_label_index]

        plt.title(f"True: {true_class_name}\nPredicted: {predicted_class_name}")

        y_text_offset = 90
        for k, class_name in enumerate(loaded_class_names):
            probability = image_predictions[k] * 100
            text = f"{class_name}: {probability:.2f}%"
            plt.text(10, y_text_offset, text, color='green', fontsize=10, weight='bold')
            y_text_offset += 20

        plt.axis("off")

plt.tight_layout()
plt.show()